In [18]:
import numpy as np
import pandas as pd
import time
from dobbel import dobbellogger
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [19]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2] * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1 / 2
    w = np.cos(absq)
    sinabs = np.sin(absq)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])


def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

def quaternion_conjugate(q):
    w, x, y, z = q
    return np.array([w, -x, -y, -z])


# Actual Rotation Fuction - Input = Acceleration vector = 1x3 , rotation_quaternion = 1*4
def rotate_vector(vector, rotation_quaternion):
    # Convert the vector to a quaternion
    vector_quaternion = np.concatenate(([0], vector))

    # Calculate the rotated quaternion
    rotated_quaternion = quat_mul(rotation_quaternion,
                                  quat_mul(vector_quaternion, quaternion_conjugate(rotation_quaternion)))

    # Extract the rotated vector from the quaternion
    rotated_vector = rotated_quaternion[1:]

    return np.array(rotated_vector)

def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [40]:
angle = np.deg2rad(-39)
rotate_y = np.array([[np.cos(angle),0,np.sin(angle)], [0,1,0],[-np.sin(angle),0,np.cos(angle)]])
trace = np.trace(rotate_y)
q_0 = np.sqrt(1+trace)/2
q_2 = 1/(4*q_0) * (np.sin(angle)+np.sin(angle))
q_rotate = np.array([q_0,0,q_2,0])
print(q_rotate)

[[ 0.77714596  0.         -0.62932039]
 [ 0.          1.          0.        ]
 [ 0.62932039  0.          0.77714596]]
[ 0.94264149  0.         -0.33380686  0.        ]


In [72]:
x_means_list = []
y_means_list = []
z_means_list = []
x_gyro_list = []
y_gyro_list = []
z_gyro_list = []

#rotated accelerometer lists
xr_acc_list = []
yr_acc_list = []
zr_acc_list = []

for i in range(3):
    i=i+1
    print("Leg de dobbelsteen met nummer ", i, " boven")
    dob = dobbellogger()
    dob.log(0.5, 200, 8, 125)
    dob.download()
    data = dob.datadf
    data = remove_nan(data)

    x_acc = data['x_acc']
    y_acc = data['y_acc']
    z_acc = data['z_acc']
    
    # Calculate the  mean
    x_mean = np.mean(data['x_acc'])
    y_mean = np.mean(data['y_acc'])
    z_mean = np.mean(data['z_acc'])
    
    # Append the mean to the list
    x_means_list.append(x_mean)
    y_means_list.append(y_mean)
    z_means_list.append(z_mean)
    
    # Create a DataFrame from the list of means
    means_df = pd.DataFrame({'x_mean': x_means_list, 'y_mean': y_means_list, 'z_mean': z_means_list})
    means = np.array([x_mean,y_mean,z_mean])
  
    # Calculate the  mean
    x_gyro = np.mean(data['x_gyro'])
    y_gyro = np.mean(data['y_gyro'])
    z_gyro = np.mean(data['z_gyro'])

    # Append the mean to the list
    x_gyro_list.append(x_gyro)
    y_gyro_list.append(y_gyro)
    z_gyro_list.append(z_gyro)

    # Create a DataFrame from the list of means
    gyro_df = pd.DataFrame({'x_gyro': x_gyro_list, 'y_gyro': y_gyro_list, 'z_gyro': z_gyro_list})

    #rotate the means to the initial triad
    init = rotate_vector(means, q_rotate)

    #append the rotated means to the list
    xr_acc_list.append(init[0])
    yr_acc_list.append(init[1])
    zr_acc_list.append(init[2])
    
    #Create a DataFrame from the list of rotated means
    init_df = pd.DataFrame({'xr_acc': xr_acc_list, 'yr_acc': yr_acc_list, 'zr_acc': zr_acc_list})

print(means_df)
print(init_df)
print(gyro_df)

Leg de dobbelsteen met nummer  1  boven
Logging data for 0.5s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  2  boven
Logging data for 0.5s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  3  boven
Logging data for 0.5s
Done! The data is located in self.datadf
     x_mean    y_mean    z_mean
0 -0.650241 -0.035717 -0.824671
1 -0.034077  0.981861 -0.029833
2  0.754264 -0.019119 -0.660251
     xr_acc    yr_acc    zr_acc
0  0.013650 -0.035717 -1.050100
1 -0.007708  0.981861 -0.044630
2  1.001683 -0.019119 -0.038437
     x_gyro    y_gyro    z_gyro
0 -0.125452  0.464463  0.038810
1 -0.139159  0.448288  0.044044
2 -0.136275  0.463099  0.053844


In [75]:
#Calibration algorithm
x_gyro_bias = np.mean(gyro_df['x_gyro'])
y_gyro_bias = np.mean(gyro_df['y_gyro'])
z_gyro_bias = np.mean(gyro_df['z_gyro'])
gyro_bias = np.array([x_gyro_bias,y_gyro_bias,z_gyro_bias])
print(gyro_bias)

[-0.1336287   0.4586165   0.04556584]


In [74]:
#remove the gravity for the bias
xr_bias_acc_list = np.array([-xr_acc_list[0],-xr_acc_list[1],1-xr_acc_list[2]])
xr_bias_acc = np.mean(xr_bias_acc_list)

yr_bias_acc_list = np.array([-yr_acc_list[0],1-yr_acc_list[1],-yr_acc_list[2]])
yr_bias_acc = np.mean(yr_bias_acc_list)

zr_bias_acc_list = np.array([1+zr_acc_list[0],-zr_acc_list[1],-zr_acc_list[2]])
zr_bias_acc = np.mean(zr_bias_acc_list)

bias = np.array([xr_bias_acc, yr_bias_acc, zr_bias_acc])
print(bias)

[-0.00254135  0.02432477  0.01098907]
